### Limpieza y transformacion de datos:

En este notebook se dara a conocer el proceso de **limpieza y transformacion de datos** que se realizara para el proyecto. Se explicara la base de datos a utilizar, los cambios a realizar en el dataframe y el resultado final para poder empezar la exploracion de datos.

### Liberias:


In [101]:
import pandas as pd
import numpy as np
import os

#### 1. Variables Historicas Completas

##### 1.1 Precipitación Acumulada

In [125]:
url_1 = "https://climatologia.meteochile.gob.cl/application/historico/aguaCaidaHistoricaMensual/330020"

tables_1 = pd.read_html(url_1, header=None)

df_preci = tables_1[2]
df_preci.drop(index=0, inplace=True)
df_preci.columns = df_preci.iloc[0]
df_preci = df_preci[1:].reset_index(drop=True)
df_preci = df_preci.drop(columns=['Superávit'])

# Limpio los datos
cols_meses = ['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic', 'Anual']
df_preci[cols_meses] = df_preci[cols_meses].replace({'s/p': 0.0, '.': np.nan })
df_preci['Años'] = df_preci['Años'].astype(int)
df_preci[cols_meses] = df_preci[cols_meses].astype(float)
df_preci.info()
df_preci.head(4)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Años    112 non-null    int32  
 1   Ene     60 non-null     float64
 2   Feb     61 non-null     float64
 3   Mar     61 non-null     float64
 4   Abr     61 non-null     float64
 5   May     61 non-null     float64
 6   Jun     61 non-null     float64
 7   Jul     60 non-null     float64
 8   Ago     61 non-null     float64
 9   Sep     61 non-null     float64
 10  Oct     61 non-null     float64
 11  Nov     61 non-null     float64
 12  Dic     59 non-null     float64
 13  Anual   61 non-null     float64
dtypes: float64(13), int32(1)
memory usage: 11.9 KB


1,Años,Ene,Feb,Mar,Abr,May,Jun,Jul,Ago,Sep,Oct,Nov,Dic,Anual
0,2025,0.0,0.0,5.4,0.5,61.3,71.2,23.3,46.0,15.3,3.6,4.3,NaN,226.6
1,2024,0.0,2.8,0.0,0.2,112.4,183.1,0.0,80.6,5.9,10.0,0.0,0.0,395.0
2,2023,0.0,0.0,0.0,8.6,5.0,51.9,45.7,89.8,76.8,4.5,28.4,0.0,310.7
3,2022,0.0,0.1,0.0,28.8,0.0,27.5,69.0,23.1,8.1,0.0,4.3,0.1,161.0


##### 1.2 Olas de Calor Históricas

In [122]:
url_2 = "https://climatologia.meteochile.gob.cl/application/historico/indiceClimaticoTendencia/330020/148"

tables_2 = pd.read_html(url_2, header=None)

df_olas_calor = tables_2[0]

df_olas_calor.drop(index=0, inplace=True)
df_olas_calor.drop(columns=2, inplace=True)
df_olas_calor.columns = df_olas_calor.iloc[0]
df_olas_calor = df_olas_calor[1:].reset_index(drop=True)
df_olas_calor.drop(df_olas_calor.tail(1).index, inplace=True)
df_olas_calor['Año'] = df_olas_calor['Año'].astype(int)
df_olas_calor['Valor (casos)'] = df_olas_calor['Valor (casos)'].astype(float)
df_olas_calor.info()
df_olas_calor.head(4)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Año            64 non-null     int32  
 1   Valor (casos)  64 non-null     float64
dtypes: float64(1), int32(1)
memory usage: 900.0 bytes


1,Año,Valor (casos)
0,1961,0.0
1,1962,0.0
2,1963,0.0
3,1964,0.0


#### 2. Variables Mensuales con Iteración (Loop de Años y Meses)

In [104]:
# HAgo una funcion para procesar los dos siguientes ya que tienen parecido formato de URL
start_year = 1970
end_year = 2024

def fecha_loop_data(variable_dato, start_year, end_year):
    df_obtenido = pd.DataFrame()
    for year in range(start_year, end_year +1):
        for mes in range(1,13):
            url = f"https://climatologia.meteochile.gob.cl/application/mensual/{variable_dato}/330020/{year}/{mes}"

            try:
                tables = pd.read_html(url, header=None)

                if tables:
                    df = tables[0].copy()
                    df.columns = ['_'.join(col).strip() for col in df.columns.values]
                    
                    df_clean = df[~df.iloc[:, 0].isin(['Mínimo', 'Máximo', 'Media Aritmética', 'Media Climatológica', 'Resumen Mensual'])].copy()
                    df_clean.insert(0, 'Año', year)
                    df_clean.insert(1, 'Mes', mes)

                    df_obtenido = pd.concat([df_obtenido, df_clean], ignore_index=True)
            except ValueError:
                pass
    return df_obtenido

# 2.1 Humedad Relativa (humedadMensual) y 2.2 Presión Atmosférica (presionQfeMensual)
temp_df_humedad = fecha_loop_data("humedadMensual", start_year,end_year)
temp_df_presion = fecha_loop_data("presionQfeMensual", start_year, end_year)

In [105]:
# Falta transformar y limpiar los datos

def transformacion_agrupacion(df, media_column, new_column):

    df[media_column] = pd.to_numeric(df[media_column], errors='coerce')
    df_mensual = df.groupby(['Año', 'Mes'])[media_column].mean().reset_index()

    df_mensual = df_mensual.rename(columns={media_column: new_column})
    df_mensual = df_mensual[['Año', 'Mes', new_column]].sort_values(by=['Año', 'Mes']).reset_index(drop=True)

    ## AHORA transforma el formato de la tabla como la de Presipitaciones (Hice esto mismo en mi tarea 4 pero sin pivot)

    df_mensual_2 = df_mensual.pivot(index='Año', columns='Mes', values=new_column).reset_index()
    df_mensual_2 = df_mensual_2.rename(columns={1:'Ene', 2:'Feb', 3: 'Mar', 4:'Abr', 5:'May', 
                    6:'Jun', 7:'Jul',8:'Ago', 9:'Sep',10:'Oct',11:'Nov',12:'Dic'})
    return df_mensual_2
df_humedad = transformacion_agrupacion(temp_df_humedad, 'Aritmética_Media', 'Humedad_Media_Mensual')
df_presion = transformacion_agrupacion(temp_df_presion, 'Aritmética_Media', 'Presion_Media_Mensual')

In [ ]:
display(df_humedad.head(4))
display(df_presion.head(4))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Año     55 non-null     int64  
 1   Ene     55 non-null     float64
 2   Feb     55 non-null     float64
 3   Mar     55 non-null     float64
 4   Abr     55 non-null     float64
 5   May     55 non-null     float64
 6   Jun     55 non-null     float64
 7   Jul     55 non-null     float64
 8   Ago     55 non-null     float64
 9   Sep     55 non-null     float64
 10  Oct     55 non-null     float64
 11  Nov     55 non-null     float64
 12  Dic     55 non-null     float64
dtypes: float64(12), int64(1)
memory usage: 5.7 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Año     55 non-null     int64  
 1   Ene     55 non-null     float64
 2   Feb     55 non-null     float64
 3 

Mes,Año,Ene,Feb,Mar,Abr,May,Jun,Jul,Ago,Sep,Oct,Nov,Dic
0,1970,51.025000,52.675862,57.009375,62.074194,74.796875,80.883871,83.184375,76.937500,71.267742,66.043750,54.487097,51.978125
1,1971,47.550000,56.934483,63.403226,63.645161,72.062500,84.161290,79.421875,77.021875,73.896774,70.425000,58.622581,56.628125
2,1972,48.193750,48.620000,55.843750,68.580645,79.821875,87.487097,82.978125,82.500000,80.287097,71.471875,65.787097,55.056250
3,1973,55.440625,66.217241,66.706250,72.129032,77.793750,79.658065,82.884375,76.287500,69.700000,70.821875,58.980645,53.959375


Mes,Año,Ene,Feb,Mar,Abr,May,Jun,Jul,Ago,Sep,Oct,Nov,Dic
0,1970,953.940625,953.203448,953.771875,954.251613,956.656250,958.435484,958.040625,958.468750,956.287097,955.500000,955.680645,953.390625
1,1971,953.468750,953.237931,954.493548,955.819355,956.740625,957.654839,957.175000,958.678125,957.283871,956.028125,954.832258,952.821875
2,1972,952.040625,952.826667,953.768750,954.954839,955.559375,955.561290,956.737500,956.465625,955.777419,956.681250,955.348387,953.787500
3,1973,952.656250,953.865517,954.050000,955.138710,956.378125,958.358065,957.871875,959.206250,958.112903,956.646875,955.022581,955.512500


#### 3. Variables Anuales con Iteración (Loop de Años)

In [128]:
def year_loop_data(variable_dato, start_year, end_year):
    df_maximos = pd.DataFrame()
    df_media = pd.DataFrame()
    df_minimos = pd.DataFrame()
    for year in range(start_year, end_year +1):
        url = f"https://climatologia.meteochile.gob.cl/application/anual/{variable_dato}/330020/{year}"
        try:
            tables = pd.read_html(url)
            
            if tables:
                df = tables[0].copy()
                df = df.iloc[32:].reset_index()
                df.drop(columns='index', inplace=True)
                
                ## Quedaran Meses_{nombre_mes} => al final los renombrare
                df.columns = ['_'.join(col).strip() for col in df.columns.values]
                # Extraigo las filas
                media_fila = df[df['Día_Día'] == 'Media Parcial'].iloc[0:1].copy()
                minimo_fila = df[df['Día_Día'] == 'Mínimo Mensual'].iloc[0:1].copy()
                maximo_fila = df[df['Día_Día'] == 'Máximo Mensual'].iloc[0:1].copy()

                for fila, tipo_fila in [(media_fila, 'Media'), (minimo_fila, 'Minimo'), (maximo_fila, 'Maximo')]:
                    fila.drop(columns=['Día_Día'])
                    fila.insert(0, 'Año' , year)

                    # startwith revisa desde el inicio del str o palabra
                    columns_meses = list(filter(lambda col: col.startswith('Meses_'), fila.columns))
                    
                    # Transformo y limpio valores
                    fila[columns_meses] = fila[columns_meses].replace('.', np.nan)
                    fila[columns_meses] = fila[columns_meses].apply(pd.to_numeric, errors='coerce')

                    if tipo_fila == 'Media':
                        df_media = pd.concat([df_media, fila], ignore_index=True)
                    elif tipo_fila == 'Minimo':
                        df_minimos = pd.concat([df_minimos, fila], ignore_index=True)
                    elif tipo_fila == 'Maximo':
                        df_maximos = pd.concat([df_maximos, fila], ignore_index=True)
        except ValueError:
            print("ERROR")
        except Exception as e:
            print(f"Error inesperado para {variable_dato} en {year}: {e}")
    df_media = df_media.rename(columns={'Meses_Enero':'Ene', 'Meses_Febrero':'Feb', 'Meses_Marzo': 'Mar', 
                    'Meses_Abril':'Abr', 'Meses_Mayo':'May', 'Meses_Junio':'Jun', 
                    'Meses_Julio':'Jul','Meses_Agosto':'Ago', 'Meses_Septiembre':'Sep',
                    'Meses_Octubre':'Oct','Meses_Noviembre':'Nov','Meses_Diciembre':'Dic'})
    df_minimos = df_minimos.rename(columns={'Meses_Enero':'Ene', 'Meses_Febrero':'Feb', 'Meses_Marzo': 'Mar', 
                    'Meses_Abril':'Abr', 'Meses_Mayo':'May', 'Meses_Junio':'Jun', 
                    'Meses_Julio':'Jul','Meses_Agosto':'Ago', 'Meses_Septiembre':'Sep',
                    'Meses_Octubre':'Oct','Meses_Noviembre':'Nov','Meses_Diciembre':'Dic'})
    df_maximos = df_maximos.rename(columns={'Meses_Enero':'Ene', 'Meses_Febrero':'Feb', 'Meses_Marzo': 'Mar', 
                    'Meses_Abril':'Abr', 'Meses_Mayo':'May', 'Meses_Junio':'Jun', 
                    'Meses_Julio':'Jul','Meses_Agosto':'Ago', 'Meses_Septiembre':'Sep',
                    'Meses_Octubre':'Oct','Meses_Noviembre':'Nov','Meses_Diciembre':'Dic'})
    return [df_media, df_minimos, df_maximos]

list_dataframes= year_loop_data("indiceUvbMaximoAnual", 2006, 2025)
df_rad_uv_media = list_dataframes[0].drop(columns={'Día_Día'})
df_rad_uv_minimo = list_dataframes[1].drop(columns={'Día_Día'})
df_rad_uv_maximo = list_dataframes[2].drop(columns={'Día_Día'})

cols_meses = ['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic']
for df in [df_rad_uv_media, df_rad_uv_minimo, df_rad_uv_maximo]:
    # Para rellenar los Nan, hare interpolacion (Usa los valores siguentes o anteriores en la columna)
    df.interpolate(method='linear', inplace=True)
    df[cols_meses] = df[cols_meses].astype(float)

C:\Users\Otros\AppData\Local\Temp\ipykernel_13872\1832433167.py:13: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df.drop(columns='index', inplace=True)
C:\Users\Otros\AppData\Local\Temp\ipykernel_13872\1832433167.py:30: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fila[columns_meses] = fila[columns_meses].replace('.', np.nan)
C:\Users\Otros\AppData\Local\Temp\ipykernel_13872\1832433167.py:30: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fila[columns_meses] = fila

In [129]:
display(df_rad_uv_media.head(4))
display(df_rad_uv_minimo.head(4))
df_rad_uv_maximo.info()

,Año,Ene,Feb,Mar,Abr,May,Jun,Jul,Ago,Sep,Oct,Nov,Dic
0,2006,NaN,10.0,8.0,5.0,3.0,2.0,2.0,4.0,5.0,7.0,10.0,12.0
1,2007,11.0,10.0,7.0,4.0,2.0,2.0,2.0,3.0,5.0,7.0,10.0,11.0
2,2008,11.0,10.0,7.0,4.0,3.0,2.0,2.0,3.0,5.0,7.0,10.0,11.0
3,2009,11.0,10.0,8.0,5.0,3.0,2.0,2.0,3.0,5.0,7.0,9.0,11.0


,Año,Ene,Feb,Mar,Abr,May,Jun,Jul,Ago,Sep,Oct,Nov,Dic
0,2006,NaN,7.0,6.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,7.0,9.0
1,2007,7.0,4.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,4.0,6.0,9.0
2,2008,10.0,8.0,6.0,1.0,1.0,1.0,1.0,1.0,2.0,3.0,1.0,10.0
3,2009,9.0,3.0,5.0,3.0,1.0,1.0,1.0,1.0,1.0,3.0,5.0,11.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Año     20 non-null     int64  
 1   Ene     19 non-null     float64
 2   Feb     20 non-null     float64
 3   Mar     20 non-null     float64
 4   Abr     20 non-null     float64
 5   May     20 non-null     float64
 6   Jun     20 non-null     float64
 7   Jul     20 non-null     float64
 8   Ago     20 non-null     float64
 9   Sep     20 non-null     float64
 10  Oct     20 non-null     float64
 11  Nov     20 non-null     float64
 12  Dic     20 non-null     float64
dtypes: float64(12), int64(1)
memory usage: 2.2 KB


### 4. Cargar los datos a la carpeta de datos

In [109]:
direct_max_uvb = os.path.join("data_temperatura", "maximo_uvb_santiago.csv")
direct_min_uvb = os.path.join("data_temperatura", "minimo_uvb_santiago.csv")
direct_med_uvb = os.path.join("data_temperatura", "media_uvb_santiago.csv")

df_rad_uv_media.to_csv(direct_med_uvb, index=False)
df_rad_uv_minimo.to_csv(direct_min_uvb, index=False)
df_rad_uv_maximo.to_csv(direct_max_uvb, index=False)

In [131]:
direct_precip = os.path.join("data_temperatura", "precipitaciones_mensuales_santiago.csv")
direct_humedad = os.path.join("data_temperatura", "humedad_mensual_santiago.csv")
direct_presion = os.path.join("data_temperatura", "presion_mensual_santiago.csv")
direct_olas_calor = os.path.join("data_temperatura", "num_anual_olas_calor_santiago.csv")

df_olas_calor.to_csv(direct_olas_calor, index=False)
df_preci.to_csv(direct_precip, index=False)
df_humedad.to_csv(direct_humedad, index=False)
df_presion.to_csv(direct_presion, index=False)